In [2]:
%pip install pettingzoo[classic]

  Using cached pettingzoo-1.24.3-py3-none-any.whl.metadata (8.5 kB)
  Using cached numpy-2.2.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached chess-1.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached rlcard-1.0.5.tar.gz (251 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pygame-2.3.0.tar.gz (12.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [95 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.24.2-VC.zip 19b1e9f5bcf700a402bb924c0194c96596283c32
      Unzipping :prebuilt_downloads\SDL2-devel-2.24.2-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://github.com/libsdl-org/SDL_ttf/releases/download/release-2.20.1/SDL2_ttf-devel-2.20.1-VC.zip 371606aceba450384428fd2852f73d2f6290b136
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.20.1-VC.zip:
      Downloading... https://github.com/libsdl-org/SDL_mixer/rele

In [12]:
from tensorflow import keras

In [ ]:
from keras.layers import Dense, Dropout

# Creación del modelo
model = keras.models.Sequential([
    Dense(64, activation = 'relu', input_shape = (72,)), # Capa Input de 72 neuronas (estado del juego)
    Dense(64, activation = 'relu'),
    Dropout(0.3),
    Dense(4, activation = 'softmax') # Capa Output de 4 neuronas (Acción a tomar) - Utilizamos softmax para que la suma de todas las probabilidades sea 1
])

In [ ]:
# Función para sacar la mejor acción a realizar (Para jugar el turno)
#   Argumentos: Estado del juego, acciones legales

# Implementación
#   Se le pasa a la capa Input del modelo el estado
#   El modelo devuelve la predicción probabilísitca de cada acción que cree que hay que tomar
#   Eliminar acciones ilegales (Multiplicar cada acción por su valor en la máscara)
#   Normalizar el resto de acciones para que sumen 1
#   Seleccionar una acción en base a una temperatura o algo así

In [3]:
from pettingzoo.classic import texas_holdem_v4
import numpy as np

In [11]:
env = texas_holdem_v4.env(num_players=3)
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    print(observation)

    print(len(observation['observation']))

    if termination or truncation:
        action = None
    else:
        mask = observation["action_mask"]
        # this is where you would insert your policy
        action = env.action_space(agent).sample(mask)

    env.step(action)
env.close()

{'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0.], dtype=float32), 'action_mask': array([1, 1, 1, 0], dtype=int8)}
72
{'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0.], dtype=float32), 'action_mask': array([1, 1, 1, 0], dtype=int8)}
72
{'observation': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0

In [ ]:
# Definir la acción basada en el estado actual
def elegir_accion(estado, acciones_disponibles, q_table, epsilon):
    # Explorar o explotar según el valor de epsilon
    if np.random.rand() < epsilon:  # Exploración
        return np.random.choice(acciones_disponibles)
    else:  # Explotación
        # Si el estado no está en q_table, inicializar
        if estado not in q_table:
            q_table[estado] = {accion: 0 for accion in acciones_disponibles}
        q_values = q_table[estado]
        return acciones_disponibles[np.argmax(q_values)]  # Elegir la acción con mejor Q-valor

In [ ]:
# Crear el entorno de Póker Texas Hold'em
env = texas_holdem_v4.env(num_players=3)
env.reset()

# Inicializar la tabla Q
q_table = {}

# Parámetros de aprendizaje
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.7  # Probabilidad de exploración
num_episodios = 100  # Número de episodios

# Entrenar el modelo
for episodio in range(num_episodios):
    env.reset()
    recompensa_total = 0
    for agente in env.agent_iter():
      resultado, recompensa, done, _, info = env.last()

      # Extraer el estado y la máscara de acciones
      estado = resultado['observation']  # Estado actual del entorno
      acciones = resultado['action_mask']  # Acciones legales para realizar
      acciones_disponibles = [i for i, legal in enumerate(acciones) if legal]

      if done:
        env.step(None)
        continue

      # Serializar estado
      estado_serializado = str(estado)

      # Inicializar en la Q-table si no existe
      if estado_serializado not in q_table:
          q_table[estado_serializado] = {accion: 0 for accion in acciones_disponibles}

      # Elegir acción
      accion = elegir_accion(estado_serializado, acciones_disponibles, q_table, epsilon)
      # Tomar acción
      env.step(accion)

      # Asegurar que la acción está en el diccionario (prevención de errores)
      if accion not in q_table[estado_serializado]:
          q_table[estado_serializado][accion] = 0

      # Obtener el nuevo estado
      resultado, _, done, _, _ = env.last()
      nuevo_estado = resultado['observation']
      nuevo_estado_serializado = str(nuevo_estado)

      nuevas_acciones = resultado['action_mask']
      nuevas_acciones_dsiponibles = [i for i, legal in enumerate(nuevas_acciones) if legal]

      if nuevo_estado_serializado not in q_table:
          q_table[nuevo_estado_serializado] = {accion: 0 for accion in nuevas_acciones_dsiponibles}

      # Actualizar la Q-table
      if not q_table[nuevo_estado_serializado]:
        max_q_siguiente = 0
      else:
        max_q_siguiente = max(q_table[nuevo_estado_serializado].values()) if not done else 0
      valor = alpha * (recompensa + gamma * max_q_siguiente - q_table[estado_serializado][accion])
      print(f"Recompensa: {recompensa}, Max Q siguiente: {max_q_siguiente}, Q actual: {q_table[estado_serializado][accion]}, Valor calculado: {valor}")
      q_table[estado_serializado][accion] += valor

      recompensa_total += recompensa

    print(f'Episodio {episodio+1}, recompensa total: {recompensa_total}')
    #break

Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Episodio 1, recompensa total: 0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Episodio 2, recompensa total: 0
Recompensa: 0, Max Q siguiente: 0, Q ac

In [ ]:
# Evaluar el modelo
estado = env.reset()
estado_tuple = tuple(estado[0]['obs'])
recompensa_total = 0
done = False
while not done:
    acciones_disponibles = list(range(env.action_num))
    accion = accion_estado(estado_tuple, q_table, 0, acciones_disponibles)  # Sin exploración (epsilon=0)
    estado, recompensa, done, _ = env.step(accion)
    estado_tuple = tuple(estado[0])
    recompensa_total += recompensa[0]

print(f'Recompensa final tras evaluación: {recompensa_total}')